In [3]:
import numpy as np
from flask import Flask, request, make_response
import json
import pickle
from flask_cors import cross_origin

In [4]:
app = Flask(__name__)

In [5]:
model = pickle.load(open('mnbmodel.pkl', 'rb'))

In [8]:
@app.route('/webhook', methods=['POST'])
@cross_origin()

def webhook(): 
    req = request.get_json(silent=True, force=True)
    res = processRequest(req)
    res = json.dumps(res, indent=4)
    r = make_response(res)
    r.headers['Content-Type'] = 'application/json'
    return r

In [9]:
def processRequest(req):
    user_symptoms = Symptom(req)
    Top3diseasepred = predictDisease(user_symptoms)
    res = makeWebhookResult(Top3diseasepred)
    return res

In [10]:
def Symptom(req):
    user_symptoms=req.get("result").get("symptoms")
    user_symptoms=list(user_symptoms)
    return user_symptoms

In [11]:
def predictDisease(user_symptoms):
    symptom=np.zeros([526],dtype=float)
    Alldiseases=model.classes_.tolist()
    
    indexes=[]
    for i in range(len(model_symptoms)):
        if model_symptoms[i] in user_symptoms:
            indexes.append(i)
    for i in indexes:
        symptom[i]=1
    top3=[]
    probab=model.predict_proba([symptom]).tolist()  
    for j in range(3):
        max=-10000000000
        h=0
        for i in range(len(probab[0])):
            if probab[0][i]>max:
                max=probab[0][i]
                h=i 
    k=[]
    k.append(l[h])
    k.append(probab[0][h])
    top3.append(k)
    probab[0][h]=-1 
    return top3


In [12]:
def makeWebhookResult(Top3diseasepred):
    result=""
    for i in Top3diseasepred:
        result+="Probability of "+str(i[0])+"is "+"{:2.3f}".format((i[1]*100))+'%'+'\n'
    return result

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jul/2020 18:15:55] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [21/Jul/2020 18:15:57] "GET /favicon.ico HTTP/1.1" 404 -
